#Profiling
Sometimes computing the likelihood is not as fast as we would like. Theano provides handy profiling tools, which pymc3 provides a wrapper `model.profile` which returns a `ProfileStats` object. Here we'll profile the likelihood and gradient for the stochastic volatility example.

First we build the model.

In [3]:
import numpy as np
from pymc3 import *
from pymc3.distributions.timeseries import *

n = 400
returns = np.genfromtxt(get_data_file('pymc3.examples', "data/SP500.csv"))[-n:]

model = Model()
with model:
    sigma = Exponential('sigma', 1. / .02, testval=.1)

    nu = Exponential('nu', 1. / 10)

    s = GaussianRandomWalk('s', sigma ** -2, shape=n)

    r = T('r', nu, lam=exp(-2 * s), observed=returns)
    


Then call profile and summarize it.

In [4]:
model.profile(model.logpt).summary()


Function profiling
  Message: /Users/fonnescj/GitHub/pymc3/pymc3/model.py:197
  Time in 1000 calls to Function.__call__: 6.456351e-02s
  Time in Function.fn.__call__: 3.473592e-02s (53.801%)
  Time in thunks: 2.467275e-02s (38.215%)
  Total compile time: 3.600923e+00s
    Number of Apply nodes: 26
    Theano Optimizer time: 9.634581e-01s
       Theano validate time: 2.970219e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 2.587607e+00s
       Import time 5.669570e-02s

Time in all call to theano.grad() 2.964401e-02s
Class
---
<% time> <sum %> <apply time> <time per call> <type> <#call> <#apply> <Class name>
  85.4%    85.4%       0.021s       1.50e-06s     C    14000      14   theano.tensor.elemwise.Elemwise
   5.8%    91.2%       0.001s       3.59e-07s     C     4000       4   theano.tensor.elemwise.Sum
   3.0%    94.2%       0.001s       3.76e-07s     C     2000       2   theano.tensor.subtensor.Subtensor
   2.4%    96.7%       0.001s       3.00e-07s     C  

In [5]:
model.profile(gradient(model.logpt, model.vars)).summary()

Function profiling
  Message: /Users/fonnescj/GitHub/pymc3/pymc3/model.py:197
  Time in 1000 calls to Function.__call__: 1.570358e-01s
  Time in Function.fn.__call__: 1.222701e-01s (77.861%)
  Time in thunks: 9.528661e-02s (60.678%)
  Total compile time: 5.955295e+00s
    Number of Apply nodes: 55
    Theano Optimizer time: 8.904371e-01s
       Theano validate time: 1.370955e-02s
    Theano Linker time (includes C, CUDA code generation/compiling): 4.988965e+00s
       Import time 1.071918e-01s

Time in all call to theano.grad() 1.577589e+00s
Class
---
<% time> <sum %> <apply time> <time per call> <type> <#call> <#apply> <Class name>
  47.2%    47.2%       0.045s       2.25e-05s     Py    2000       2   theano.tensor.basic.Split
  31.9%    79.1%       0.030s       1.12e-06s     C    27000      27   theano.tensor.elemwise.Elemwise
   4.7%    83.8%       0.005s       6.45e-07s     C     7000       7   theano.tensor.elemwise.Sum
   4.5%    88.3%       0.004s       2.14e-06s     C     2000 